In [1]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [2]:
import sys
sys.path.insert(0,"./../")

In [3]:
import os
import dotenv
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
from textblob import TextBlob
from pypdf import PdfReader
import nltk
from nltk.corpus import stopwords

In [4]:
dotenv.load_dotenv()

True

In [5]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nelsonlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
pdf_path = "../documents/GIS+Prospectus+UK.pdf"

In [7]:
pdf_reader = PdfReader(pdf_path)

In [8]:
pdf_page_parse_dict_list = []

In [9]:
for page_num, page in enumerate(pdf_reader.pages):
     text = page.extract_text()
     pdf_page_parse_dict = {}
     pdf_page_parse_dict['page_num'] = page_num
     pdf_page_parse_dict['text'] = text
     pdf_page_parse_dict_list.append(pdf_page_parse_dict)


In [10]:
df_page  = pd.DataFrame(pdf_page_parse_dict_list)

In [11]:
pdf_sentence_dict_list =[]
for page_dict in  tqdm(pdf_page_parse_dict_list):
     page_num = page_dict['page_num']
     text = page_dict['text']

     blob = TextBlob(text)
     for index,sentence in enumerate(blob.sentences):
          sentence_dict = {}
          sentence_dict['page_num'] = page_num
          sentence_dict['sentence_index'] = index
          sentence_dict['sentence'] = str(sentence)
          pdf_sentence_dict_list.append(sentence_dict)

100%|██████████| 882/882 [00:00<00:00, 3040.86it/s]


In [12]:
df_sentence  = pd.DataFrame(pdf_sentence_dict_list)

### (1) Contains fund ?

In [13]:
# def contains_fund(sentence):
#     sentence = sentence.lower()
#     if "fund" in sentence:
#         return True
#     return False

In [14]:
# df_sentence['contains_fund'] = df_sentence['sentence'].apply(contains_fund)

In [15]:
"sdf asdf asdf".capitalize()

'Sdf asdf asdf'

In [16]:
def extract_fund_name(text):
    import re

    pattern = r'\b(?:[^a-zA-Z]*([A-Z][a-zA-Z ]+Fund)[^a-zA-Z]*)\b'
    matches = re.findall(pattern, text, re.IGNORECASE)

    filters = []
    for name in matches:
        if "  " in name:
            continue
        if name == " ".join([w.capitalize() for w  in name.split(" ")]):
            filters.extend([n.lstrip() + "Fund" for n in name.split("Fund")])

    return filters

In [17]:
df_sentence['fund_names'] = df_sentence['sentence'].apply(extract_fund_name)

In [18]:
all_names = []
for names in df_sentence[df_sentence['fund_names'].apply(lambda x: len(x)!=0)]['fund_names']:
    all_names.extend(names)


In [19]:
all_names = list(set(all_names))

In [20]:
all_names.sort(key = lambda x: len(x))

In [21]:
merged_names = all_names.copy() 
for (index,name) in enumerate(all_names):
    for next_name in all_names[index+1:]:
        if name in next_name:
            all_names[index]=""
            continue

In [22]:
all_names =  [name for name in all_names if name]

In [23]:
stop_fund_names = set(word.capitalize() + " Fund"  for word in stopwords.words('english'))

In [24]:
all_names =  [name for name in all_names if name not in stop_fund_names]

In [25]:
all_names

['Term Fund',
 'Asset Fund',
 'Small Fund',
 'Europe Fund',
 'Markets Fund',
 'Reserve Fund',
 'Original Fund',
 'Selected Fund',
 'Dividend Fund',
 'Euro Bond Fund',
 'Reporting Fund',
 'Underlying Fund',
 'Affiliated Fund',
 'Global Bond Fund',
 'Call Risk A Fund',
 'Euro Credit Fund',
 'State Street Fund',
 'Compensation Fund',
 'Dynamic Bond Fund',
 'Long Duration Fund',
 'Global Advantage Fund',
 'Futures Strategy Fund',
 'Strategic Income Fund',
 'Euro Income Bond Fund',
 'Total Return Bond Fund',
 'Currency Hedging A Fund',
 'Term Opportunities Fund',
 'Global Real Return Fund',
 'Diversified Income Fund',
 'Emerging Local Bond Fund',
 'Term Local Currency Fund',
 'Low Duration Income Fund',
 'Low Average Duration Fund',
 'Factor Europe Equity Fund',
 'Commodity Real Return Fund',
 'Energy Infrastructure Fund',
 'Emerging Markets Bond Fund',
 'Mortgage Opportunities Fund',
 'Global High Yield Bond Fund',
 'International Monetary Fund',
 'Euro Long Average Duration Fund',
 'Low D